# 4. Bayesian Networks

In [1]:
import numpy as np
import pandas as pd
import lark

import pybn
import pybn.examples
import pybn.reader.oobn

from IPython.display import display, HTML

print(f'Using PyBN version: {pybn.__version__}')
print(f'Using Pandas version: {pd.__version__}')

%load_ext autoreload
%autoreload 2

Using PyBN version: 2019-07-02 14:31
Using Pandas version: 0.24.1


In [2]:
# Load the Student Network from the examples
Gstudent = pybn.examples.get_student_network()
Gstudent

<BayesianNetwork name='Student'>
  <Node RV='I' />
  <Node RV='S' />
  <Node RV='D' />
  <Node RV='G' />
  <Node RV='L' />
</BayesianNetwork>

In [3]:
# Accessing individual Nodes/CPTs is easy
Gstudent['S'].cpt

S,s0,s1
I,,
i0,0.95,0.05
i1,0.20,0.80


In [4]:
# The Node-property 'states' will return a list of allowed states.
Gstudent['S'].states

['s0', 's1']

In [5]:
# Querying the network is also simple. To compute the probability of G given I:
Gstudent.compute_posterior(['I'], {'G': 'g1'})

{'I': factor(I)
 I
 i0    0.38674
 i1    0.61326
 dtype: float64}